# Extract and Plot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = "pgc-mdd2025_no23andMe_div_v3-49-46-01.tsv.gz"

# ----------------------------------------------------
# 1. Define expected columns manually (based on PGC MDD GWAS schema)
# ----------------------------------------------------
column_names = [
    "CHR",        # Chromosome
    "BP",         # Base-pair position
    "SNP",        # rsID
    "A1",         # Effect allele
    "A2",         # Other allele
    "BETA",       # Effect size
    "SE",         # Standard error
    "PVAL",       # p-value
    "FRQ_A1",     # Frequency of A1 in cases
    "FRQ_U",      # Frequency in controls
    "INFO",       # Imputation info
    "N",          # Sample size or effective N
    "NMISS",      # Missing count
    "N_TOTAL",    # Total samples
    "QC",         # QC flag
    "GENPOS",     # Genetic position
    "RSQ"         # Imputation R² or similar
]

# Load with manual headers
df = pd.read_csv(
    file_path,
    sep='\t',
    compression='gzip',
    names=column_names,
    comment='#',
    header=None,
    low_memory=False
)

print(f"✅ Loaded {len(df):,} variants with proper headers.")
print(df.head())

# ----------------------------------------------------
# 2. Filter significant SNPs (p < 5e-8)
# ----------------------------------------------------
sig_threshold = 5e-8
significant_snps = df[df['PVAL'] < sig_threshold].copy()

print(f"\n🧬 Significant SNPs (p < {sig_threshold}): {len(significant_snps):,}")

# ----------------------------------------------------
# 3. (Optional) Manhattan plot
# ----------------------------------------------------
df['-log10(P)'] = -np.log10(df['PVAL'])
plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=df.sample(min(500000, len(df))),  # sample for faster plotting
    x='BP',
    y='-log10(P)',
    hue='CHR',
    palette='tab20',
    s=5,
    linewidth=0
)
plt.axhline(-np.log10(sig_threshold), color='red', linestyle='--', label='Genome-wide significance')
plt.legend()
plt.title('Manhattan Plot - Depression GWAS (PGC MDD)')
plt.xlabel('Chromosome Position (BP)')
plt.ylabel('-log10(p-value)')
plt.tight_layout()
plt.show()

# ----------------------------------------------------
# 4. Save significant SNPs
# ----------------------------------------------------
significant_snps.to_csv("significant_snps_depression.tsv", sep='\t', index=False)
print("\n💾 Saved significant SNPs to 'significant_snps_depression.tsv'")


# Mapping SNPs to Genes

In [8]:
import pandas as pd
import requests
import time

# Load your SNP dataset
df = pd.read_csv("significant_snps_depression.tsv", sep='\t')
print(f"Loaded {len(df)} SNPs")

# Function to map SNP position to nearest gene via Ensembl
def get_nearest_gene(chr_num, bp):
    server = "https://rest.ensembl.org"
    ext = f"/overlap/region/human/{chr_num}:{bp}-{bp}?feature=gene"
    headers = {"Content-Type": "application/json"}
    try:
        r = requests.get(server + ext, headers=headers, timeout=10)
        if not r.ok:
            return None
        data = r.json()
        if len(data) == 0:
            return None
        # Get closest gene symbol
        nearest_gene = data[0].get('gene_id', None)
        gene_symbol = data[0].get('external_name', None)
        return gene_symbol
    except Exception:
        return None

# Map a subset first (you can scale up later)
results = []
for i, row in df.iterrows():  # try all snps
    gene = get_nearest_gene(row['CHR'], int(row['BP']))
    results.append({'SNP': row['SNP'], 'symbol': gene})
    if i % 10 == 0:
        print(f"Processed {i+1} SNPs...")
    time.sleep(0.3)

mapped = pd.DataFrame(results)
merged = df.merge(mapped, on='SNP', how='left')
merged.to_csv("depression_SNP_gene_positional.tsv", sep='\t', index=False)

print(f"✅ Saved mapped SNP-gene pairs: depression_SNP_gene_positional.tsv")
print(merged.head(10))


Loaded 13074 SNPs
Processed 1 SNPs...
Processed 11 SNPs...
Processed 21 SNPs...
Processed 31 SNPs...
Processed 41 SNPs...
Processed 51 SNPs...
Processed 61 SNPs...
Processed 71 SNPs...
Processed 81 SNPs...
Processed 91 SNPs...
Processed 101 SNPs...
Processed 111 SNPs...
Processed 121 SNPs...
Processed 131 SNPs...
Processed 141 SNPs...
Processed 151 SNPs...
Processed 161 SNPs...
Processed 171 SNPs...
Processed 181 SNPs...
Processed 191 SNPs...
Processed 201 SNPs...
Processed 211 SNPs...
Processed 221 SNPs...
Processed 231 SNPs...
Processed 241 SNPs...
Processed 251 SNPs...
Processed 261 SNPs...
Processed 271 SNPs...
Processed 281 SNPs...
Processed 291 SNPs...
Processed 301 SNPs...
Processed 311 SNPs...
Processed 321 SNPs...
Processed 331 SNPs...
Processed 341 SNPs...
Processed 351 SNPs...
Processed 361 SNPs...
Processed 371 SNPs...
Processed 381 SNPs...
Processed 391 SNPs...
Processed 401 SNPs...
Processed 411 SNPs...
Processed 421 SNPs...
Processed 431 SNPs...
Processed 441 SNPs...
Pro

# Gene Enrichment

In [12]:
import pandas as pd
from gprofiler import GProfiler

# Load mapped SNP-gene file
df = pd.read_csv("depression_SNP_gene_positional.tsv", sep='\t')

# Filter non-null genes
gene_list = df['symbol'].dropna().unique().tolist()
print(f"Unique genes mapped: {len(gene_list)}")

# Initialize GProfiler
gp = GProfiler(return_dataframe=True)
results = gp.profile(
    organism='hsapiens',
    query=gene_list,
    sources=['GO:BP', 'GO:MF', 'KEGG']
)

# Save full results
results.to_csv("depression_pathway_enrichment.csv", index=False)
print("✅ Enrichment results saved as depression_pathway_enrichment.csv")

# Show top pathways
print(results[['native', 'name', 'p_value', 'intersection_size']].head(10))


Unique genes mapped: 456
✅ Enrichment results saved as depression_pathway_enrichment.csv
       native                                name   p_value  intersection_size
0  GO:0007399          nervous system development  0.000086                 74
1  GO:0007417  central nervous system development  0.000570                 39
2  GO:0048731                  system development  0.003246                 96
3  GO:0035608             protein deglutamylation  0.003722                  4
4  GO:0007275  multicellular organism development  0.005399                107
5  GO:0048699               generation of neurons  0.005910                 47
6  GO:0022008                        neurogenesis  0.011905                 51
7  GO:0030182              neuron differentiation  0.016101                 44
8  GO:0048856    anatomical structure development  0.021340                126
9  GO:0007420                   brain development  0.025632                 28
